In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/HIV_Resistance/GSE14542'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Circulating Monocytes in HIV-1 infected Viremic Subjects Exhibit an Anti-Apoptotic Gene Signature"
!Series_summary	"Mechanisms that may allow circulating monocytes to persist as CD4 T cells diminish in HIV-1 infection have not been investigated. We have characterized steady-state gene expression signatures in circulating monocytes from HIV-infected subjects and have identified a stable anti-apoptosis gene signature comprised of 38 genes associated with p53, CD40L, TNF and MAPKinase signaling networks. The significance of this gene signature is indicated by our demonstration of cadmium chloride- or Fas ligand-induced apoptosis resistance in circulating monocytes in contrast to increasing apoptosis in CD4 T cells from the same infected subjects. As potential mechanisms in vivo, we show that monocyte CCR5 binding by HIV-1 virus or agonist chemokines serve as independent viral and host modulators resulting in increased monocyte apoptosis resistance in

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = True
trait_row = 0
age_row = None
gender_row = None

# Define conversion functions
def convert_trait(value):
    # Assuming the HIV Resistance is a binary indicator derived from Viral Load: detectable or not
    value = value.split(':')[1].strip()
    return 1 if int(value.replace(',', '')) > 0 else 0

def convert_age(value):
    # Placeholder function as age_row is None
    return None

def convert_gender(value):
    # Placeholder function as gender_row is None
    return None

# Save cohort information
save_cohort_info('GSE14542', './preprocessed/HIV_Resistance/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'HIV_Resistance', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/HIV_Resistance/trait_data/GSE14542.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM363812': [0], 'GSM363813': [0], 'GSM363814': [0], 'GSM363815': [0], 'GSM363816': [0], 'GSM363817': [0], 'GSM363818': [0], 'GSM363819': [0], 'GSM363820': [0], 'GSM363821': [0], 'GSM363822': [0], 'GSM363823': [0], 'GSM363824': [1], 'GSM363825': [1], 'GSM363826': [1], 'GSM363827': [1], 'GSM363828': [1], 'GSM363829': [1], 'GSM363830': [1], 'GSM363831': [1], 'GSM363832': [1], 'GSM363833': [1], 'GSM363834': [1], 'GSM363835': [1], 'GSM363836': [1], 'GSM363837': [0], 'GSM363838': [0], 'GSM363839': [0], 'GSM363840': [0], 'GSM363841': [1], 'GSM363842': [1], 'GSM363843': [1], 'GSM363844': [1]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1', '2', '3', '4', '5'], 'filter': ['Ha05', 'Ha05', 'Ha05', 'Ha05', 'Ha05'], 'GB_ACC': ['AA128826', 'T71887', 'AA478043', 'AA677706', 'R92124'], 'name': ['arylsulfatase B', 'apolipoprotein C-IV', 'interferon regulatory factor 1', 'lactotransferrin', 'MRE11 meiotic recombination 11 homolog A (S. cerevisiae)'], 'symbol': ['ARSB', 'APOC4', 'IRF1', 'LTF', 'MRE11A'], 'SPOT_ID': [nan, nan, nan, nan, nan]}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Specify the identifiers key and gene symbol key based on the gene annotation data
identifier_key = 'ID'
gene_symbol_key = 'symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes
gene_mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping and obtain the gene expression dataframe
gene_data = apply_gene_mapping(gene_data, gene_mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/HIV_Resistance/gene_data/GSE14542.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'HIV_Resistance')

# 4. Save the cohort information.
save_cohort_info('GSE14542', './preprocessed/HIV_Resistance/cohort_info.json', True, True, trait_biased, unbiased_merged_data)

# 5. If the trait is not severely biased, save the merged data to a csv file.
if not trait_biased:
    csv_path = './preprocessed/HIV_Resistance/GSE14542.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'HIV_Resistance', the least common label is '0.0' with 16 occurrences. This represents 48.48% of the dataset.
The distribution of the feature 'HIV_Resistance' in this dataset is fine.

